# Python Offchain Attest

In [ ]:
from eth_abi.abi import encode
import json

# Label Pool Schema v1.0.0
schema = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68'

# Input the label in OLI format here:
#   chain_id should follow the CAIP-2 standard
#   contract_address should be the address of the contract you want to label
#   tags_json should be a JSON string
chain_id = 'eip155:8453'
contract_address = '0x498581ff718922c3f8e6a244956af099b2652b2b'
tags_json = json.dumps({
                'is_eoa': False, 
                'is_proxy': False,
                'contract_name': 'Pool Manager v4',
                'deployment_tx': '0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349',
                'deployer_address': '0x2179a60856E37dfeAacA0ab043B931fE224b27B6',
                'owner_project': 'uniswap',
                'deployment_date': '2025-01-21 20:28:43',
                'version': 4,
                'source_code_verified': 'https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/'
            })

# ABI encode your label data
encoded_data = encode(['string', 'string'], [chain_id, tags_json])
data = f"0x{encoded_data.hex()}"

print(data)

0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000086569703a3834353300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001b37b2269735f656f61223a2066616c73652c202269735f70726f7879223a2066616c73652c2022636f6e74726163745f6e616d65223a2022506f6f6c204d616e61676572207634222c20226465706c6f796d656e745f7478223a2022307832356634383266626439346364656131316230313837333265343535623865396139343062393333636162646533633063356464363365613635653835333439222c20226465706c6f7965725f61646472657373223a2022307832313739613630383536453337646665416163413061623034334239333166453232346232374236222c20226f776e65725f70726f6a656374223a2022756e6973776170222c20226465706c6f796d656e745f64617465223a2022323032352d30312d32312032303a32383a3433222c202276657273696f6e223a20342c2022736f757263655f636f64655f7665726966696564223a202268747470

In [ ]:
import eth_account.messages
from web3 import Web3
import secrets
import time
import eth_account
from eth_keys import keys

# Connect to an Ethereum node
rpc = 'https://sepolia.base.org' # in production use: 'https://mainnet.base.org'
rpc_chain_number = 84532 # in production use 8453
web3 = Web3(Web3.HTTPProvider(rpc))
if not web3.is_connected():
    raise Exception("Failed to connect to the Ethereum node")

# Import your account to sign the attestation with
address = '...' #### fill in here!
private_key = '...' #### fill in here!

# Create a random salt (32 bytes)
salt = f"0x{secrets.token_hex(32)}"

# Current time in seconds
current_time = int(time.time())

# Typed data for the attestation
typed_data = {
    "version": 2,
    "recipient": contract_address,
    "time": current_time,
    "revocable": True, # default to True
    "schema": schema,
    "refUID": "0x0000000000000000000000000000000000000000000000000000000000000000", # default we do not have a reference UID
    "data": data,
    "expirationTime": 0, # default does not expire
    "salt": salt,
}

# EIP-712 typed data format
types = {
    "domain": {
        "name": "EAS Attestation",
        "version": "1.2.0",
        "chainId": rpc_chain_number,
        "verifyingContract": "0x4200000000000000000000000000000000000021" # EAS Attestation contract address (might be different for other networks than Base/Base-Sepolia)
    },
    "primaryType": "Attest",
    "message": typed_data,
    "types": {
        "Attest": [
            {
                "name": "version",
                "type": "uint16"
            },
            {
                "name": "schema",
                "type": "bytes32"
            },
            {
                "name": "recipient",
                "type": "address"
            },
            {
                "name": "time",
                "type": "uint64"
            },
            {
                "name": "expirationTime",
                "type": "uint64"
            },
            {
                "name": "revocable",
                "type": "bool"
            },
            {
                "name": "refUID",
                "type": "bytes32"
            },
            {
                "name": "data",
                "type": "bytes"
            },
            {
                "name": "salt",
                "type": "bytes32"
            }
        ]
    }
}

# sign message 
msg = {
    "types": {
        "EIP712Domain": [],
        "Type":  []
       },
       "domain": types.get("domain"),
       "primaryType": 'Type',
       "message": typed_data
}

# Convert the hex private key to the proper key object
if private_key.startswith('0x'):
    private_key = private_key[2:]
private_key = keys.PrivateKey(bytes.fromhex(private_key))

# Make sure to install correct version of eth-account (pip install eth-account==0.13.5)
# Sign the typed data
account = eth_account.Account.from_key(private_key)
signed_message = account.sign_typed_data(
    domain_data=types["domain"],
    message_types=types["types"],
    message_data=typed_data
)
print('signed message: 0x' + signed_message.signature.hex())

# function to calculate the UID for offchain attestation (v2)
def calculate_attestation_uid_v2(schema, recipient, attester, timestamp, data, expiration_time=0, revocable=True, ref_uid="0x0000000000000000000000000000000000000000000000000000000000000000", bump=0, salt=f"0x{secrets.token_hex(32)}"):
    # Version
    version = 2
    version_bytes = version.to_bytes(version, byteorder='big')
    # First, add '0x' prefix if not present
    if schema.startswith('0x'):
        schema_as_string = schema
    else:
        schema_as_string = '0x' + schema
    # convert the hex string to a string while not decoding it as actual hex bytes
    schema_utf8_bytes = schema_as_string.encode('utf-8')
    schema_bytes = schema_utf8_bytes
    # Conversions all values to bytes
    recipient_bytes = Web3.to_bytes(hexstr=recipient)
    attester_bytes = Web3.to_bytes(hexstr=attester)
    timestamp_bytes = timestamp.to_bytes(8, byteorder='big')
    expiration_bytes = expiration_time.to_bytes(8, byteorder='big')
    revocable_bytes = bytes([1]) if revocable else bytes([0])
    ref_uid_bytes = Web3.to_bytes(hexstr=ref_uid)
    data_bytes = Web3.to_bytes(hexstr=data)
    salt_bytes = Web3.to_bytes(hexstr=salt)
    bump_bytes = bump.to_bytes(4, byteorder='big')
    # Pack all values
    packed_data = (version_bytes + schema_bytes + recipient_bytes + attester_bytes + timestamp_bytes + expiration_bytes + revocable_bytes + ref_uid_bytes + data_bytes + salt_bytes + bump_bytes)
    # Calculate keccak256 hash
    uid = Web3.keccak(packed_data)
    return uid

attester = '0x0000000000000000000000000000000000000000' # for offchain UID calculation, the attester is always null
uid = calculate_attestation_uid_v2(schema, contract_address, attester, int(typed_data["time"]), typed_data["data"], salt=salt)
uid_hex = '0x' + uid.hex()

print(f"UID: {uid_hex}")

signed message: 0x9aef6031f36f0ec75037d09f32ec46300ae06898515e37c61bc906c21475d846335ee841b2996ca96d99a126f06f24f55548d0fe2e4b21efd89e29a1541415a21b
UID: 0xe7b496e67f0ead6258ae692abbd1158fe52a270c2a69e444ef318edc9f361be6


In [ ]:
import requests

# API endpoint URL
api_url = "https://base-sepolia.easscan.org/offchain/store" # in production use: "https://base.easscan.org/offchain/store"

# Format the attestation & send to API post endpoint
pkg = {
    "sig": {
        "domain": types["domain"],
        "primaryType": types["primaryType"],
        "types": types["types"],
        "message": types["message"],
        "uid": uid_hex,
        "version": 2,
        "signature": {
            "r": hex(signed_message.r),
            "s": hex(signed_message.s),
            "v": signed_message.v
        }
    },
    "signer": address
}

# convert sig.message.time, sig.message.expirationTime, and sig.domain.chainId to strings
pkg["sig"]["message"]["time"] = str(pkg["sig"]["message"]["time"])
pkg["sig"]["message"]["expirationTime"] = str(pkg["sig"]["message"]["expirationTime"])
pkg["sig"]["domain"]["chainId"] = str(pkg["sig"]["domain"]["chainId"])

# Prepare pkg for the API endpoint
payload = {
    "filename": "OLI.txt",
    "textJson": json.dumps(pkg, separators=(',', ':'))
}
headers = {
    "Content-Type": "application/json"
}
print('payload is:', payload)

# Post the data to the API
try:
    response = requests.post(api_url, json=payload, headers=headers)
    # Print the response
    print("Status Code:", response.status_code)
    print("JSON Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Error sending data to API: {e}")

payload is: {'filename': 'OLI.txt', 'textJson': '{"sig":{"domain":{"name":"EAS Attestation","version":"1.2.0","chainId":"84532","verifyingContract":"0x4200000000000000000000000000000000000021"},"primaryType":"Attest","types":{"Attest":[{"name":"version","type":"uint16"},{"name":"schema","type":"bytes32"},{"name":"recipient","type":"address"},{"name":"time","type":"uint64"},{"name":"expirationTime","type":"uint64"},{"name":"revocable","type":"bool"},{"name":"refUID","type":"bytes32"},{"name":"data","type":"bytes"},{"name":"salt","type":"bytes32"}]},"message":{"version":2,"recipient":"0x498581ff718922c3f8e6a244956af099b2652b2b","time":"1741774833","revocable":true,"schema":"0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68","refUID":"0x0000000000000000000000000000000000000000000000000000000000000000","data":"0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000

# Python Offchain Revoke
##### as of now the only way to revoke is to use an onchain transaction, see `2_label_pool/tooling_write/bulk_onchain_python`